In [ ]:
!python --version

Python 3.11.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

#!pip install -r "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/requirements.txt"
!pip install decord
!pip install pytorchvideo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 63.4 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188686 sha256=ea947fdb670d67f74c22fb20799649151b6ddf5164ad7c49dc653c8daf7b58c3
  Stored in directory: /root/.cache/pip/wheels/a4/6d/ae/d016375a73be141a0e11bb42289e2d0b046c35687fc8010ecc
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=ec2b6535cff4f14f7fbb4469792e240a0c67ddd33a106e00be2b96af6bc251a0
  Stored in directory: /root/.cache/pip/whee

In [ ]:
import io
import os
import json
import argparse
import numpy as np
import torch
from decord import cpu, VideoReader, bridge
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
json_filename = "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/inputs/labeled_top_50_queries.json"
clip_folder = "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/inputs/re_clips"
output_json = "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/outputs/CogVLM2_labeled_top_50_results.json"

In [ ]:
MODEL_PATH = "THUDM/cogvlm2-llama3-caption"

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

parser = argparse.ArgumentParser(description="CogVLM2-Video CLI Demo")
parser.add_argument('--quant', type=int, choices=[4, 8], help='Enable 4-bit or 8-bit precision loading', default=0)
args = parser.parse_args([])

In [ ]:
def load_video(video_data, strategy='chat'):
    bridge.set_bridge('torch')
    mp4_stream = video_data
    num_frames = 24
    decord_vr = VideoReader(io.BytesIO(mp4_stream), ctx=cpu(0))

    frame_id_list = None
    total_frames = len(decord_vr)
    if strategy == 'base':
        clip_end_sec = 60
        clip_start_sec = 0
        start_frame = int(clip_start_sec * decord_vr.get_avg_fps())
        end_frame = min(total_frames,
                        int(clip_end_sec * decord_vr.get_avg_fps())) if clip_end_sec is not None else total_frames
        frame_id_list = np.linspace(start_frame, end_frame - 1, num_frames, dtype=int)
    elif strategy == 'chat':
        timestamps = decord_vr.get_frame_timestamp(np.arange(total_frames))
        timestamps = [i[0] for i in timestamps]
        max_second = round(max(timestamps)) + 1
        frame_id_list = []
        for second in range(max_second):
            closest_num = min(timestamps, key=lambda x: abs(x - second))
            index = timestamps.index(closest_num)
            frame_id_list.append(index)
            if len(frame_id_list) >= num_frames:
                break

    video_data = decord_vr.get_batch(frame_id_list)
    video_data = video_data.permute(3, 0, 1, 2)
    return video_data

In [ ]:
# prompt: open this file: /usr/local/lib/python3.11/dist-packages/pytorchvideo/transforms/augmentations.py and change the line: import torchvision.transforms.functional_tensor as F_t to import torchvision.transforms.functional as F_t and check to see if changes are made

filepath = '/usr/local/lib/python3.11/dist-packages/pytorchvideo/transforms/augmentations.py'

# Read the content of the file
with open(filepath, 'r') as file:
    file_content = file.read()

# Replace the specific line
old_line = 'import torchvision.transforms.functional_tensor as F_t'
new_line = 'import torchvision.transforms.functional as F_t'
file_content = file_content.replace(old_line, new_line)

# Write the modified content back to the file
with open(filepath, 'w') as file:
    file.write(file_content)

# Check if the changes were made by reading the file again and searching for the new line
with open(filepath, 'r') as file:
    updated_content = file.read()

if new_line in updated_content:
    print(f"Successfully changed '{old_line}' to '{new_line}' in {filepath}")
else:
    print(f"Failed to change '{old_line}' to '{new_line}' in {filepath}")

# Optional: Print the first few lines of the modified file to visually confirm
print("\nFirst 10 lines of the modified file:")
with open(filepath, 'r') as file:
    for i in range(10):
        print(file.readline().strip())


Successfully changed 'import torchvision.transforms.functional_tensor as F_t' to 'import torchvision.transforms.functional as F_t' in /usr/local/lib/python3.11/dist-packages/pytorchvideo/transforms/augmentations.py

First 10 lines of the modified file:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.

"""Video transforms that are used for advanced augmentation methods."""

from typing import Any, Callable, Dict, Optional, Tuple

import torch
import torchvision
import torchvision.transforms.functional as F_t
from torchvision.transforms.functional import InterpolationMode


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True
).eval().to(DEVICE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

...

model-00006-of-00006.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [ ]:
def predict(prompt, video_data, temperature=0.1):
    strategy = 'chat'

    video = load_video(video_data, strategy=strategy)

    history = []
    query = prompt
    inputs = model.build_conversation_input_ids(
        tokenizer=tokenizer,
        query=query,
        images=[video],
        history=history,
        template_version=strategy
    )
    inputs = {
        'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
        'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
        'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
        'images': [[inputs['images'][0].to('cuda').to(TORCH_TYPE)]],
    }
    gen_kwargs = {
        "max_new_tokens": 2048,
        "pad_token_id": 128002,
        "top_k": 1,
        "do_sample": False,
        "top_p": 0.1,
        "temperature": temperature,
    }
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

In [ ]:
# === PROCESS ALL CLIPS ===
results = []
with open(json_filename, 'r') as f:
    nlq_entries = json.load(f)

for entry in nlq_entries:
    clip_uid = entry["clip_uid"]
    query_idx = entry["query_idx"]
    question = entry["question"]
    gt_answer = entry["answer"]
    video_path = os.path.join(clip_folder, f"{clip_uid}_{query_idx}.mp4")
    print(video_path)

    with open(video_path, 'rb') as video_file:
        video_data = video_file.read()  # Read the file content into bytes
        model_answer = predict(question, video_data, temperature=0.1)

    results.append({
        "clip_uid": clip_uid,
        "query_idx": query_idx,
        "question": question,
        "gt_answer": gt_answer,
        "model_answer": model_answer
    })
    print(f"✅ {clip_uid}_{query_idx}")

/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/8d3f5b12-ac2c-4315-80ed-bb827aa91bd4_0.mp4


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ 8d3f5b12-ac2c-4315-80ed-bb827aa91bd4_0
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/c664f078-9b34-4a58-b949-180ac4bc0980_4.mp4


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ c664f078-9b34-4a58-b949-180ac4bc0980_4
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/c7873f07-7b9c-4794-9a56-fdebeeceee68_6.mp4


...

✅ 47c24350-cc10-4ec6-8407-24f778981a82_0
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/ed0a0e94-c79b-462d-a64b-238f26fd6fc6_6.mp4


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ ed0a0e94-c79b-462d-a64b-238f26fd6fc6_6
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/b1bd830a-a738-4ebe-9e88-c348def0de90_4.mp4


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ b1bd830a-a738-4ebe-9e88-c348def0de90_4


In [ ]:
# === SAVE OUTPUT ===
with open(output_json, "w") as f:
    json.dump(results, f, indent=2)

print(f"🎉 Done. Results saved to: {output_json}")

🎉 Done. Results saved to: /content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/labeled_top_50_results.json
